In [1]:
import numpy as np
import arrayfire as af
import pylab as pl
af.set_backend("cpu")
%matplotlib inline

In [2]:
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20  
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8     
pl.rcParams['xtick.minor.size'] = 4     
pl.rcParams['xtick.major.pad']  = 8     
pl.rcParams['xtick.minor.pad']  = 8     
pl.rcParams['xtick.color']      = 'k'     
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'    

pl.rcParams['ytick.major.size'] = 8     
pl.rcParams['ytick.minor.size'] = 4     
pl.rcParams['ytick.major.pad']  = 8     
pl.rcParams['ytick.minor.pad']  = 8     
pl.rcParams['ytick.color']      = 'k'     
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in'  

In [3]:
# Setting velocity and spatial grid points
N_positions = 10000
N_velocity  = 10000
ghost_zones = 3

In [4]:
# Boundaries of domain
left_boundary  = 0
right_boundary = 1.0
length         = right_boundary - left_boundary

In [5]:
# Setting mass of the particle, boltzmann-constant
mass_particle      = 1.0
boltzmann_constant = 1.0

In [6]:
# Scattering time scale
tau   = 0.05
# Magnitude of maximum velocity
v_max = 5.0

In [7]:
# Time Parameters for the simulation:
dt         = 0.005 # Size of the time-step
final_time = 1.0
time       = np.arange(dt, (final_time + dt), dt)

In [8]:
# Setting up of spatial and velocity grids:
x  = np.linspace(left_boundary, right_boundary, N_positions)
dx = x[1] - x[0]

In [9]:
# Obtaining the coordinates for the ghost-zones:
x_ghost_left  = np.linspace(-(ghost_zones)*dx + left_boundary, left_boundary - dx, ghost_zones)
x_ghost_right = np.linspace(right_boundary + dx, right_boundary + ghost_zones*dx , ghost_zones)

In [10]:
# Combining them to obtain the entire spatial grid
x  = np.concatenate([x_ghost_left, x, x_ghost_right])

In [11]:
# Obtaining the velocity grid
v  = np.linspace(-v_max, v_max, N_velocity)

# Converting the arrayfire's native format
x  = af.to_array(x)
v  = af.to_array(v)

In [12]:
# Conversion to allow for easy vectorization
x = af.tile(x, 1, N_velocity)
v = af.tile(af.reorder(v), N_positions + 2*ghost_zones, 1)

In [13]:
def calculate_density(f, v):
    deltav           = af.sum(v[0, 1]-v[0, 0])
    value_of_density = af.sum(f, 1)*deltav
    af.eval(value_of_density)
    return(value_of_density)

In [14]:
def calculate_momentum(f, v):
    deltav            = af.sum(v[0, 1]-v[0, 0])
    value_of_momentum = af.sum(f*v, 1)*deltav
    af.eval(value_of_momentum)
    return(value_of_momentum)

In [15]:
def calculate_temperature(f, v):
    deltav               = af.sum(v[0, 1]-v[0, 0])
    value_of_temperature = af.sum(f*v**2, 1)*deltav
    value_of_temperature = value_of_temperature/calculate_density(f, v)
    af.eval(value_of_temperature)
    return(value_of_temperature)

In [16]:
def f_MB(x, v, f):
    n = af.tile(calculate_density(f, v), 1, N_velocity)
    T = af.tile(calculate_temperature(f, v), 1, N_velocity)
    f_MB = n*af.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T))*\
             af.exp(-mass_particle*v**2/(2*boltzmann_constant*T))
    af.eval(f_MB)
    return(f_MB)

In [17]:
def f_interp(dt, x, v, f):
    x_new     = x - v*dt
    step_size = af.sum(x[1,0] - x[0,0])
    f_inter   = af.constant(0, N_positions + 2*ghost_zones, N_velocity)
    f_inter   = af.Array.as_type(f_inter, af.Dtype.f64)
    
    # Implementing periodic B.C's :
    x_new[ghost_zones:-ghost_zones, :] =\
    af.select(x_new[ghost_zones:-ghost_zones, :]<=left_boundary,
              x_new[ghost_zones:-ghost_zones, :] + length,
              x_new[ghost_zones:-ghost_zones, :]
             )
    
    x_new[ghost_zones:-ghost_zones, :] =\
    af.select(x_new[ghost_zones:-ghost_zones, :]>=right_boundary,
              x_new[ghost_zones:-ghost_zones, :] - length,
              x_new[ghost_zones:-ghost_zones, :]
             )
    
    # Interpolating:
    f_inter[ghost_zones:-ghost_zones,:] = af.approx1(f, (x_new[ghost_zones:-ghost_zones,:]/step_size), af.INTERP.CUBIC)
    
    af.eval(f_inter)
    return f_inter


In [18]:
# Intializing the values for f
f_initial = af.constant(0, N_positions + 2*ghost_zones, N_velocity)
rho       = 0.1*af.sin(2*np.pi*x) + 1

In [19]:
f_initial = rho * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*1))*\
                  af.exp(-mass_particle*v**2/(2*boltzmann_constant*1))

In [20]:
# Implementing periodic B.C's:

f_initial[:ghost_zones, :] = \
f_initial[N_positions - 1:N_positions - 1 + ghost_zones, :]

f_initial[ghost_zones + N_positions:, :] = \
f_initial[1 + ghost_zones:1 + 2*ghost_zones, :]

In [21]:
# Initializing the starting value for the simulation
f_current   = f_initial
data        = np.zeros(time.size + 1)
data_rho    = np.zeros(time.size + 1)
data_rho[0] = af.max(af.abs(calculate_density(f_initial, v)))
data[0]     = af.sum(af.abs(calculate_momentum(f_initial, v)))

In [22]:
for time_index, t0 in enumerate(time):
    
    # print("Computing For Time Index = ", time_index)
    # print("Physical Time            = ", t0)
    # print() # To leave a blank line.
    
    # We shall split the Boltzmann-Equation and solve it:
    
    # In this step we are solving the collisionless equation
    
    f_current = af.Array.as_type(f_current, af.Dtype.f64)
    fstar     = f_interp(dt, x, v, f_current)
    
    # Implementing periodic B.C's
    
    fstar[:ghost_zones, :] = \
    fstar[N_positions - 1:N_positions - 1 + ghost_zones, :]

    fstar[ghost_zones + N_positions:, :] = \
    fstar[1 + ghost_zones:1 + 2*ghost_zones, :]
    
#     We turn off the term v(df/dx) for the following two steps
#     f0             = f_MB(x, v, fstar)
#     f_intermediate = fstar - (dt/2)*(fstar          - f0)/tau
#     f_new          = fstar - (dt)  *(f_intermediate - f0)/tau 

#     # Implementing periodic B.C's
    
#     f_new[:ghost_zones, :] = \
#     f_new[N_positions - 1:N_positions - 1 + ghost_zones, :]

#     f_new[ghost_zones + N_positions:, :] = \
#     f_new[1 + ghost_zones:1 + 2*ghost_zones, :]

    f_current = fstar
    
    mom                = af.abs(calculate_momentum(f_current, v))
    data[time_index+1] = af.sum(mom)
    data_rho[time_index + 1] = af.max(af.abs(calculate_density(f_current, v)))

RuntimeError: In function virtual void* cpu::MemoryManager::nativeAlloc(size_t)
In file src/backend/cpu/memory.cpp:67
Unable to allocate memory


In [ ]:
pl.plot(time, data[:time.size])
pl.ylabel('$p_{\mathrm{total}}$')
pl.xlabel('$t$')
pl.title('$\mathrm{Variation\;of\;total\;momentum\;with\;time}$')

In [ ]:
N  = 512
Nt = 100
dx = 1./N
i  = np.arange(0, N, 1)
x  = (i + 0.5)*dx
tFinal  = 1.
t  = np.linspace(0, tFinal, Nt)

amplitude   = 0.1
k           = 2*np.pi
theta0      = 1
rho0        = 1.0

def rhoInit(x):
    return rho0 + amplitude*np.sin(k*x)

def thetaInit(x):
    return theta0

thetaInit = np.vectorize(thetaInit)

In [ ]:
from scipy.integrate import quad
def f0(v, x):    
    rho   = rhoInit(x)
    theta = thetaInit(x)
    
    m = 1.
    k = 1.
        
    return rho * (m/(2*np.pi*k*theta))**(1./2.) * np.exp(-m*v**2./(2.*k*theta))

def f(v, x, t):

    return f0(v, x - v*t)

def rhoIntegrand(v, x, t):

    integralMeasure = 1.0
    
    return integralMeasure * f(v, x, t)

In [ ]:
gridPoint = N/4
solnVst = np.zeros(Nt)
for n in range(Nt):
    #print ("n = ", n)
    integral = quad(rhoIntegrand, -np.inf, np.inf, args=(x[gridPoint], t[n]))
    solnVst[n]  = integral[0]

In [ ]:
pl.plot(t, np.abs(solnVst - rho0), label='Analytical')
pl.plot(time, np.abs(data_rho[:time.size] - 1), label='Numerical')
pl.xlim([0, 0.9])
pl.ylim([0, 0.3])
pl.legend()